# Data Acquisition and Understanding

Once the APIs are built and the raw data is available its the job of data engineer to perform the data acquisition and to provide the data further to data scientist for further exploitation tasks. 

Data engineers basically work on principles of ETL, i.e., ```Extract Transform and Load```. Although it could be ELT as well. The order is not really important for us now (at this level)

Difference between ETL and ELT can be found [here](https://www.oracle.com/fr/database/elt-vs-etl.html)

In [ ]:
# Lets start by importing the libraries 
import os
import glob
import pandas as pd # library for data wrangling

We'll do here is to ```Extract``` the data coming from different sources (csv and json) of file

In [ ]:
# the path to our data files
path = "/content/drive/MyDrive/Colab_Notebooks/orness/acquisition_data/"

In [ ]:
# Listing all the csv files
for csvfile in glob.glob(path+"*.csv"):
  print(csvfile)

In [ ]:
# listing json file
for jsonfile in glob.glob(path+"*.json"):
  print(jsonfile)

In [ ]:
# Looping throught the files, turning them into a DataFrame and appending it to a list
df_list = []
for csvfile in glob.glob(path+"*.csv"):
  df_tmp = pd.read_csv(csvfile)
  df_list.append(df_tmp)
  
# Finally concatinating all the dataframes   
df_csv = pd.concat(df_list)

---------Looking at the pandas DataFrame for a brief moment -------

pd.DataFrame.iloc  
pd.DataFrame['col']  
pd.DataFrame[index]  
pd.DataFrame.shape  
pd.DataFrame.dtypes  

Follow the rich doc of pandas [here](https://pandas.pydata.org/docs/user_guide/index.html)

In [ ]:
df_csv.head(10)

In [ ]:
df_json = pd.read_json(path+'winequality_4.json', orient ='records', lines=True)

In [ ]:
df_csv.dtypes

In [ ]:
df_json.dtypes

# Data Cleaning

Why cleaning the dataframes seems to be a such a crutial part?  

Python is (luckily) not a statically typed language. That is why we can develop so fast. The drawback of a dynamically typed language is that we may run into more problems during runtime than we would using a strict static typing scheme. Python will at least throw an error during runtime

Hence we need to typecast the data before the processing or analysis.  

Advance data cleaning technique could involve: Data Validation (using pydantic and pandas), creating generators list (for infinite sequences of dataflow)

```Transform``` into the same data type

In [ ]:
# trying to convert each entry to float 
df_csv['alcohol'] = df_csv['alcohol'].astype(float)

In [ ]:
# printing the str values 
for i in df_csv['alcohol']:
  try:
    float(i)
  except:
    print(i)

In [ ]:
df_csv['alcohol'] = df_csv['alcohol'].str.replace('%', '')
df_csv['alcohol'] = df_csv['alcohol'].astype(float)

In [ ]:
# checking the null values in the dataframe
df_csv.isnull().sum()

In [ ]:
df_csv.info()

For the other json file we will parse it using the pydantic library.  
[Pydantic](https://docs.pydantic.dev/usage/validators/) enforces type hints at runtime, and provides user friendly errors when data is invalid.  



In [ ]:
from pydantic import ValidationError, Field, BaseModel

In [ ]:
class DataValidation(BaseModel):
  fixed_acidity: float
  volatile_acidity: float
  citric_acid: float
  residual_sugar: float
  chlorides: float
  free_sulfur_dioxide: float
  total_sulfur_dioxide: float
  density: float
  pH: float #= Field(ge=6, le=8, description="ph of water")
  sulphates: float
  alcohol: float
  TARGET: int

In [ ]:
dict_to_val = df_json.to_dict(orient="records")

In [ ]:
for i in dict_to_val:
  print(i)

In [ ]:
error_count = 0
verified_data = []
for i in dict_to_val:
  try:    
    verified_data.append(DataValidation(**i))
  except ValidationError as ve:
    print(f"row: {i}, error: {ve}")
    error_count += 1

In [ ]:
error_count

In [ ]:
df_json_clean = pd.DataFrame([s.__dict__ for s in verified_data])

In [ ]:
df_json_clean.dtypes

```Load``` the data to have a final data

In [ ]:
df_clean = pd.concat([df_json_clean, df_csv], ignore_index = True)

![ETL](https://drive.google.com/uc?id=19vKLoFRkSCkH7yfG4KZBs71L8jyQK0y2)